In [ ]:
from google.colab import files
uploaded = files.upload()   #NodeAttributesWithSubCommunity.csv

Saving NodeAttributesWithSubCommunity.csv to NodeAttributesWithSubCommunity.csv


In [ ]:
import pandas as pd
import numpy as np
import random
import math

In [ ]:
NodeAttributesWithSubCommunity= pd.read_csv("NodeAttributesWithSubCommunity.csv")   # T1T2CommunityCoresTrusses.csv NodeAttributesWithSubCommunity.csv   NodeAttributesWithSubCommunityGC.csv
NodeAttributesWithSubCommunity.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)   #There was an unamed column in the file, needed to delete that column
NodeAttributesWithSubCommunity.drop(["a"], axis=1, inplace=True)

print("Total nodes are:", NodeAttributesWithSubCommunity['UserID'].count())
#NodeAttributesWithSubCommunity['ComWithSubCom'] = NodeAttributesWithSubCommunity['ComWithSubCom'].astype('string')
#NodeAttributesWithSubCommunity['SubCommunity'] = NodeAttributesWithSubCommunity['SubCommunity'].astype('string')
#NodeAttributesWithSubCommunity['Community'] = NodeAttributesWithSubCommunity['Community'].astype('string')
NodeAttributesWithSubCommunity


Total nodes are: 943


,UserID,SubCommunity,Community,Age,Gender,Occupation,Zipcode,AgeGroup,State,NormalizedAge,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,...,War,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference,ComWithSubCom
0,1,0,0,24,1,technician,85711,Youth,AZ,0.328767,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0.042589,0.010221,0.666667,0.585714,0.666667,0.440000,0.694505,0.688000,0.96,0.785047,0.700000,1.000000,0.692308,0.584615,0.720000,0.786364,0.800000,0.723077,0.736000,0.733333,587.0,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0
1,2,0,1,53,0,other,94043,Senior,CA,0.726027,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0.024390,0.000000,0.760000,0.866667,0.800000,0.600000,0.762500,0.755556,0.00,0.765714,0.600000,0.900000,0.600000,0.600000,0.700000,0.825000,0.750000,0.716667,0.733333,0.000000,123.0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,10
2,3,1,0,23,1,writer,32067,Youth,FL,0.315068,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0.040984,0.000000,0.557143,0.700000,0.000000,0.000000,0.516667,0.600000,1.00,0.581818,0.000000,0.500000,0.480000,0.400000,0.636364,0.680000,0.550000,0.504762,0.560000,0.000000,122.0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1
3,4,1,0,24,1,technician,43537,Youth,OH,0.328767,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0.035714,0.000000,0.775000,0.700000,0.000000,0.000000,1.000000,0.950000,1.00,0.900000,0.000000,0.000000,0.800000,1.000000,0.800000,0.866667,0.766667,0.781818,0.900000,0.000000,56.0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1
4,5,0,1,33,0,other,15213,MiddleAge,PA,0.452055,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0.036458,0.005208,0.628571,0.648485,0.757143,0.489655,0.600000,0.777778,0.00,0.533333,0.500000,1.000000,0.507143,0.666667,0.600000,0.463158,0.703030,0.589474,0.642857,0.500000,384.0,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,0,1,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,7,2,2,57,1,administrator,91344,Senior,CA,0.780822,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0.049697,0.021818,0.776842,0.800000,0.892308,0.811111,0.765909,0.800000,0.84,0.842466,0.828571,0.763636,0.710638,0.830769,0.780952,0.771429,0.795833,0.784416,0.897561,0.788889,825.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
939,8,0,2,36,1,administrator,5201,MiddleAge,VT,0.493151,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.071429,0.019481,0.815385,0.764706,0.000000,0.266667,0.480000,0.822222,0.00,0.847059,0.000000,0.800000,1.000000,0.000000,0.600000,0.800000,0.858824,0.755556,0.872727,0.866667,154.0,1,1,0,0,0,1,0,1,0,1,1,0,1,1,1,1,1,1,20
940,9,0,3,29,1,student,1002,Youth,MA,0.397260,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.102041,0.000000,1.000000,1.000000,0.000000,0.000000,0.900000,0.000000,0.00,0.800000,0.000000,1.000000,1.000000,0.000000,0.800000,0.950000,0.950000,0.880000,0.800000,0.000000,49.0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,1,1,0,30
941,10,0,0,53,1,lawyer,90703,Senior,CA,0.726027,0,0,

In [ ]:
#Creating groups of column names
UnNormalizedFeatures=['Age', 'Occupation',	'Zipcode',	'AgeGroup',	'State','NumMoviesWatched']
NumericFeatures = ['NormalizedAge']
#AllStates = pd.unique(NodeAttributesWithSubCommunity['State']).tolist()
#AllStates = [x for x in AllStates if str(x) != 'nan']
AllOccupations = pd.unique(NodeAttributesWithSubCommunity['Occupation'])
AgeLabels = ['Teenager','Youth','MiddleAge','Senior','SuperSenior']
AllGenres= ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 
             'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller',  'War', 'Western']
AllGenresRatings = []
for genre in AllGenres:
   AllGenresRatings.append(genre+'Rating')
AllGenresPreference = []
for genre in AllGenres:
   AllGenresPreference.append(genre+'Preference') 

In [ ]:
SubCommunityCentroids = NodeAttributesWithSubCommunity.groupby(by=["ComWithSubCom"],as_index=False).mean()
SubCommunityCentroids.head()

,ComWithSubCom,UserID,SubCommunity,Community,Age,Gender,NormalizedAge,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,PA,WA,VT,...,Thriller,War,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference
0,0,113.873016,0.0,0.0,35.116402,0.888889,0.481047,0.021164,0.338624,0.502646,0.116402,0.021164,0.0,0.079365,0.010582,0.0,0.153439,0.052910,0.105820,0.015873,0.0,0.037037,0.0,0.089947,0.021164,0.0,0.121693,0.010582,0.026455,0.089947,0.0,0.100529,0.084656,0.052910,0.116402,0.021164,0.026455,0.031746,0.068783,0.0,...,0.118522,0.046862,0.005586,0.685150,0.675731,0.511322,0.571612,0.707290,0.730801,0.377302,0.759069,0.376727,0.599597,0.575611,0.556284,0.679986,0.742938,0.711738,0.715039,0.741781,0.388430,214.798942,0.809524,0.767196,0.645503,0.629630,0.862434,0.883598,0.449735,0.952381,0.380952,0.698413,0.634921,0.613757,0.851852,0.931217,0.846561,0.873016,0.878307,0.476190
1,1,124.898058,1.0,0.0,34.558252,0.878641,0.473401,0.014563,0.359223,0.529126,0.087379,0.009709,0.0,0.077670,0.009709,0.0,0.126214,0.087379,0.072816,0.019417,0.0,0.038835,0.0,0.058252,0.024272,0.0,0.199029,0.009709,0.004854,0.082524,0.0,0.063107,0.126214,0.004854,0.101942,0.033981,0.029126,0.019417,0.019417,0.0,...,0.114602,0.045021,0.005999,0.688613,0.667312,0.500313,0.561300,0.694494,0.713313,0.343438,0.744456,0.359110,0.606000,0.566694,0.552187,0.656843,0.722675,0.714006,0.697688,0.712241,0.407539,215.941748,0.766990,0.762136,0.611650,0.621359,0.805825,0.868932,0.398058,0.917476,0.373786,0.728155,0.635922,0.640777,0.800971,0.883495,0.849515,0.839806,0.825243,0.466019
2,2,66.645833,2.0,0.0,34.250000,0.895833,0.469178,0.020833,0.416667,0.437500,0.104167,0.020833,0.0,0.125000,0.000000,0.0,0.187500,0.020833,0.145833,0.041667,0.0,0.020833,0.0,0.062500,0.020833,0.0,0.145833,0.020833,0.000000,0.062500,0.0,0.062500,0.083333,0.000000,0.166667,0.020833,0.000000,0.000000,0.062500,0.0,...,0.116664,0.046574,0.005286,0.687932,0.688533,0.517387,0.562228,0.697181,0.722337,0.291627,0.759561,0.373681,0.603214,0.504409,0.524226,0.644732,0.720869,0.735408,0.702035,0.740286,0.354968,184.625000,0.791667,0.750000,0.687500,0.645833,0.854167,0.895833,0.333333,0.958333,0.375000,0.708333,0.520833,0.645833,0.833333,0.937500,0.854167,0.916667,0.916667,0.375000
3,3,308.000000,3.0,0.0,60.000000,1.000000,0.821918,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.101485,0.037129,0.011139,0.721053,0.720930,0.816667,0.744000,0.773016,0.789189,0.880000,0.739877,0.714286,0.846154,0.740741,0.757143,0.757895,0.736364,0.765000,0.748780,0.753333,0.755556,808.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,10,110.355556,0.0,1.0,35.622222,0.311111,0.487976,0.044444,0.311111,0.466667,0.177778,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.111111,0.000000,0.088889,0.111111,0.044444,0.0,...,0.093901,0.047251,0.00740

In [ ]:
CommunityLabel = 'Community'
columnsToKeep = [CommunityLabel,'Gender','NormalizedAge']
#columnsToKeep.extend(AgeLabels)
columnsToKeep.extend(AllOccupations)
columnsToKeep.extend(AllGenres)
print(columnsToKeep)
UserDemography = NodeAttributesWithSubCommunity[columnsToKeep]

UserDemographyCentroids = UserDemography.groupby(CommunityLabel).filter(lambda x: len(x) > 5).groupby(by=[CommunityLabel],as_index=False).mean()

#UserDemographyCentroids = UserDemography.groupby(by=["ComWithSubCom"],as_index=False).mean()
print(UserDemographyCentroids.shape)
UserDemographyCentroids

['Community', 'Gender', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']
(7, 42)


,Community,Gender,NormalizedAge,technician,other,writer,executive,administrator,student,lawyer,educator,scientist,entertainment,programmer,librarian,homemaker,artist,engineer,marketing,none,healthcare,retired,salesman,doctor,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,0,0.885135,0.476984,0.078829,0.0,0.103604,0.094595,0.0,0.000000,0.036036,0.0,0.083333,0.065315,0.15991,0.0,0.0,0.083333,0.144144,0.072072,0.022523,0.02027,0.013514,0.013514,0.009009,0.114467,0.060393,0.014839,0.029105,0.130174,0.043316,0.004864,0.191322,0.004721,0.010111,0.022957,0.019657,0.035168,0.086858,0.063782,0.116464,0.045955,0.005758
1,1,0.357143,0.504403,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.113030,0.059451,0.016327,0.040814,0.153481,0.034449,0.002136,0.192483,0.005705,0.006911,0.023969,0.022520,0.026382,0.099130,0.054017,0.094461,0.047542,0.006943
2,2,0.596154,0.519362,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.111473,0.055914,0.008839,0.026576,0.137756,0.042033,0.005097,0.216521,0.006556,0.009649,0.022819,0.019194,0.026415,0.098618,0.054674,0.098155,0.051586,0.008125
3,3,0.726190,0.302511,0.000000,0.0,0.000000,0.000000,0.0,0.994048,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.005952,0.00000,0.000000,0.000000,0.000000,0.132950,0.066359,0.013431,0.023896,0.123719,0.041380,0.001900,0.173743,0.004763,0.006565,0.034126,0.015804,0.027931,0.092054,0.069692,0.123509,0.044200,0.003978
4,4,0.705128,0.580436,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.106975,0.062080,0.010096,0.029020,0.130271,0.032358,0.006733,0.225280,0.006536,0.007019,0.018701,0.020754,0.026646,0.112578,0.054925,0.083983,0.055993,0.010021
5,5,0.367347,0.506011,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.111666,0.051349,0.016033,0.030278,0.146933,0.026883,0.002316,0.225736,0.006120,0.007286,0.020486,0.021622,0.020502,0.106230,0.046693,0.105600,0.047372,0.006893
6,6,0.000000,0.429550,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.169799,0.077468,0.014168,0.052239,0.114812,0.029501,0.001348,0.125388,0.008432,0.001171,0.010950,0.014168,0.047381,0.060966,0.061665,0.162135,0.048410,0.000000


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

num_users = UserDemography.shape[0]
num_centroids = UserDemographyCentroids.shape[0]

centroidNames = dict(list(zip(UserDemographyCentroids.index, UserDemographyCentroids.Community)))
#print(centroidNames)
assignedCentroids=[]
for i in range(0,num_users):
  row = UserDemography.iloc[i].to_numpy().reshape(1, -1)
  maxsimilarity = 0
  matchedcentroid = 0
  for j in range(0,num_centroids):
    comparedrow = UserDemographyCentroids.iloc[j].to_numpy().reshape(1, -1)
    sim = cosine_similarity(row,comparedrow).item()   #converting to scalar
    #print(sim)
    if (sim > maxsimilarity):
      maxsimilarity = sim
      matchedcentroid = centroidNames[j]
  assignedCentroids.append(matchedcentroid)

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

columnsToKeep = ['UserID','Community','ComWithSubCom']
MatchedCommunity = NodeAttributesWithSubCommunity[columnsToKeep]
MatchedCommunity['AssignedCommunity'] = assignedCentroids
MatchedCommunity.head()

,UserID,Community,ComWithSubCom,AssignedCommunity
0,1,0,0,0
1,2,1,10,1
2,3,0,1,0
3,4,0,1,0
4,5,1,10,1


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test = MatchedCommunity[CommunityLabel]
y_pred = MatchedCommunity['AssignedCommunity']
print(precision_score(y_test, y_pred, labels=np.unique(y_pred), average="weighted"))
print(recall_score(y_test, y_pred, labels=np.unique(y_pred), average="weighted")) 

0.9970910289350139
0.9989373007438895
